In [ ]:
import numpy as np
import pandas as pd
from resizeimage import resizeimage
from PIL import Image
from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D

# dimensions of our images.
img_width, img_height = 320, 160
RESIZE_IMAGE_WIDTH = 32
RESIZE_IMAGE_HEIGHT = 16
csv_file_path = 'data/driving_log.csv'
driving_log = pd.read_csv('data/driving_log.csv')
# this is the output labels for each frame
steering_angles = driving_log['steering']
n_train_data = len(steering_angles)

# file name of each image
image_file_names = 'data/' + driving_log['center']
print('image file name 0 is {}'.format(image_file_names[0]))
print('number of train data is {}'.format(n_train_data))
print('shape of label is {}'.format(driving_log.shape[0]))
   
# define the model with Keras
nb_filters = 32
nb_filters_2 = 16
kernel_size_w = 3
kernel_size_h = 3
dropout = 0.25
print('shape of steering_angles is {}'.format(steering_angles.shape))
input_shape = (RESIZE_IMAGE_HEIGHT, RESIZE_IMAGE_WIDTH, 3)
model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size_w, kernel_size_h, border_mode='valid', input_shape = input_shape))
model.add(Dropout(dropout))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters_2, kernel_size_w, kernel_size_h, border_mode='valid'))
model.add(Dropout(dropout))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=["accuracy"])

def resize_image(img):
    image = resizeimage.resize_width(img, RESIZE_IMAGE_WIDTH)
    return image

# Python data generator
def generate_arrays_from_file(path):
    while 1:
        for index in range(n_train_data):
            # create Numpy arrays of input data
            # and labels, using each row in the csv file
            image = Image.open(image_file_names[index])
            image = resize_image(image)
            x = img_to_array(image)
            y = np.array(steering_angles[index])

            yield x.reshape(1,RESIZE_IMAGE_HEIGHT, RESIZE_IMAGE_WIDTH,3), y.reshape(1)

generator = generate_arrays_from_file(csv_file_path)
print(next(generator))
model.fit_generator(generator, samples_per_epoch=n_train_data, nb_epoch=10)
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')  # save the weights after training or during training

image file name 0 is data/IMG/center_2016_12_01_13_30_48_287.jpg
number of train data is 8036
shape of label is 8036
shape of steering_angles is (8036,)
(array([[[[ 132.,  165.,  209.],
         [ 131.,  164.,  208.],
         [ 131.,  164.,  209.],
         ..., 
         [ 125.,  158.,  204.],
         [ 135.,  167.,  216.],
         [ 134.,  166.,  212.]],

        [[ 143.,  175.,  222.],
         [ 146.,  179.,  229.],
         [ 140.,  174.,  220.],
         ..., 
         [ 133.,  166.,  213.],
         [ 142.,  175.,  223.],
         [ 140.,  172.,  218.]],

        [[ 142.,  172.,  206.],
         [  92.,  111.,  111.],
         [ 137.,  167.,  200.],
         ..., 
         [ 143.,  176.,  219.],
         [ 151.,  185.,  229.],
         [ 155.,  189.,  235.]],

        ..., 
        [[ 126.,  117.,  105.],
         [ 142.,  125.,  112.],
         [ 145.,  125.,  111.],
         ..., 
         [ 169.,  155.,  136.],
         [ 169.,  155.,  137.],
         [ 160.,  145.,  128.]